# Deep Learning Playground

We are interested in the following features:

- H1: region-averaged time-series
- H2: region-level connectivities (from H1, optional: triu-k1)
- H3: network-averaged time-series (from H1)
- H4: network connectivity (from H3, optional: triu-k1)
- H5: networks connectivity (from H2, optional: triu-k0)

> Note that we are not going to take the upper triangular part of the connectivity matrix and full matrices are used instead.

In [5]:
%reload_ext autoreload
%autoreload 2

import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import RichProgressBar

from src.acnets.deep import ACNetsDataModule, MultiHeadModel

In [6]:
datamodule = ACNetsDataModule(atlas='dosenbach2010', kind='partial correlation', batch_size=8)
datamodule.setup()

n_regions = datamodule.train[0][0].shape[1]
n_networks = datamodule.train[0][2].shape[1]
model = MultiHeadModel(n_regions, n_networks, n_embeddings=8)

# DEBUG
trainer = pl.Trainer(accelerator='auto', max_epochs=1,
                     log_every_n_steps=1, callbacks=[RichProgressBar()])

trainer.fit(model, datamodule)
trainer.test(model, datamodule=datamodule)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


┏━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name           ┃ Type               ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ train_accuracy │ MulticlassAccuracy │      0 │
│ 1 │ val_accuracy   │ MulticlassAccuracy │      0 │
│ 2 │ x4_head        │ Sequential         │  1.7 K │
│ 3 │ x5_head        │ Sequential         │  1.7 K │
│ 4 │ cls_head       │ Sequential         │     46 │
└───┴────────────────┴────────────────────┴────────┘

Trainable params: 3.4 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 3.4 K                                                                                                
Total estimated model params size (MB): 0

Output()

`Trainer.fit` stopped: `max_epochs=1` reached.


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/accuracy       │            0.5            │
│   test/dropout_accuracy   │            0.5            │
│       test/loss_cls       │    0.6940850019454956     │
└───────────────────────────┴───────────────────────────┘

[{'test/loss_cls': 0.6940850019454956,
  'test/accuracy': 0.5,
  'test/dropout_accuracy': 0.5}]

In [28]:
from ray.train.lightning import (
    RayDDPStrategy,
    RayLightningEnvironment,
    RayTrainReportCallback,
    prepare_trainer,
)
from ray import tune
from ray.train.torch import TorchTrainer
from ray.train import RunConfig, ScalingConfig, CheckpointConfig

def train_func(config):
    datamodule = ACNetsDataModule(atlas=config['atlas'],
                                  kind=config['kind'],
                                  batch_size=config['batch_size'])
    datamodule.setup()

    n_regions = datamodule.train[0][0].shape[1]
    n_networks = datamodule.train[0][2].shape[1]
    model = MultiHeadModel(n_regions, n_networks, n_embeddings=config['n_embeddings'])

    trainer = pl.Trainer(
        strategy=RayDDPStrategy(),
        callbacks=[RayTrainReportCallback()],
        plugins=[RayLightningEnvironment()],
        enable_progress_bar=False,
        max_epochs=config['max_epochs'],
        log_every_n_steps=1
    )
    trainer = prepare_trainer(trainer)
    trainer.fit(model, datamodule=datamodule)
    trainer.test(model, datamodule=datamodule)

TorchTrainer(
    train_func,
    train_loop_config={
        'batch_size': tune.choice([32]),
        'atlas': tune.choice(['dosenbach2010']),
        'kind': tune.choice(['partial correlation']),
        'n_embeddings': tune.choice([8]),
        'max_epochs': 1,
    },
    run_config=RunConfig(
        name='acnets_multihead',
        storage_path='~/ray_results',
        checkpoint_config=CheckpointConfig(
            num_to_keep=1,
            checkpoint_score_attribute='val/accuracy',
            checkpoint_score_order='max',
        )
    ),
    scaling_config=ScalingConfig(num_workers=2, use_gpu=False)
).fit()

(TorchTrainer pid=3290) Starting distributed worker processes: ['3298 (127.0.0.1)', '3299 (127.0.0.1)']
(RayTrainWorker pid=3298) Setting up process group for: env:// [rank=0, world_size=2]
(RayTrainWorker pid=3298) [W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())
(RayTrainWorker pid=3298) GPU available: False, used: False
(RayTrainWorker pid=3298) TPU available: False, using: 0 TPU cores
(RayTrainWorker pid=3298) IPU available: False, using: 0 IPUs
(RayTrainWorker pid=3298) HPU available: False, using: 0 HPUs
(RayTrainWorker pid=3299) Missing logger folder: /Users/morteza/ray_results/acnets_multihead/TorchTrainer_0d7c6_00000_0_atlas=dosenbach2010,batch_size=32,kind=partial_correlation,n_embeddings=8_2024-01-07_19-03-25/lightning_logs
(RayTrainWorker pid=3298) 
(RayTrainWorker pid=3298)   | Name           | Type      

(RayTrainWorker pid=3298) ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
(RayTrainWorker pid=3298) ┃        Test metric        ┃       DataLoader 0        ┃
(RayTrainWorker pid=3298) ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
(RayTrainWorker pid=3298) │       test/accuracy       │           0.25            │
(RayTrainWorker pid=3298) │   test/dropout_accuracy   │           0.25            │
(RayTrainWorker pid=3298) │       test/loss_cls       │    0.7183140516281128     │
(RayTrainWorker pid=3298) └───────────────────────────┴───────────────────────────┘


2024-01-07 19:05:04,530	INFO tune.py:1047 -- Total run time: 99.19 seconds (99.16 seconds for the tuning loop).


Result(
  metrics={'train/loss_cls': 0.6864034533500671, 'train/loss_recon': 0.20026294887065887, 'train/loss': 0.8866664171218872, 'train/accuracy': 0.5833333134651184, 'val/loss_cls': 0.7183140516281128, 'val/accuracy': 0.25, 'val/dropout_accuracy': 0.25, 'epoch': 0, 'step': 1},
  path='/Users/morteza/ray_results/acnets_multihead/TorchTrainer_0d7c6_00000_0_atlas=dosenbach2010,batch_size=32,kind=partial_correlation,n_embeddings=8_2024-01-07_19-03-25',
  filesystem='local',
  checkpoint=Checkpoint(filesystem=local, path=/Users/morteza/ray_results/acnets_multihead/TorchTrainer_0d7c6_00000_0_atlas=dosenbach2010,batch_size=32,kind=partial_correlation,n_embeddings=8_2024-01-07_19-03-25/checkpoint_000000)
)

In [ ]:
atlases = [
    'dosenbach2010',
    'dosenbach2007',
    'difumo_64_2mm',
    'difumo_128_2mm',
    'difumo_256_2mm',
    'difumo_512_2mm',
    'difumo_1024_2mm',
    'gordon2014_2mm',
    'cort-maxprob-thr25-2mm',
    'seitzman2018',
    'friedman2020',
]

kinds = [
    'correlation',
    'partial correlation',
    'tangent',
    'covariance',
    'precisions'
]